**Copyright**

Jelen forráskód a Budapesti Műszaki és Gazdaságtudományi Egyetemen tartott
"Deep Learning a gyakorlatban Python és LUA alapon" tantárgy segédanyagaként készült.

A tantárgy honlapja: http://smartlab.tmit.bme.hu/oktatas-deep-learning
Deep Learning kutatás: http://smartlab.tmit.bme.hu/deep-learning

A forráskódot GPLv3 licensz védi. Újrafelhasználás esetén lehetőség szerint kérjük
az alábbi szerzőt értesíteni.

2019 (c) Csapó Tamás Gábor (csapot kukac tmit pont bme pont hu),
Gyires-Tóth Bálint, Zainkó Csaba



Links:

[keras-tuner] https://github.com/keras-team/keras-tuner

[blog post] https://www.mikulskibartosz.name/using-keras-tuner-to-tune-hyperparameters-of-a-tensorflow-model/

In [0]:
# az óra második felében a keras-tuner-t vizsgáljuk,
# ami 2019-ben vált elérhetővé (https://twitter.com/fchollet/status/1189992078991708160?lang=en),
# és most is aktívan fejlesztik,
# a legújabb commit-ok néhány naposak (https://github.com/keras-team/keras-tuner/commits/master)
# de mindenképp érdemes foglalkozni vele,
# mert nagy erőkkel dolgozik rajta a keras csapata

# egyelőre viszonylag kevés tutorial van róla, de ezek pl hasznosak voltak:
# 1) https://www.mikulskibartosz.name/using-keras-tuner-to-tune-hyperparameters-of-a-tensorflow-model/
# 2) https://www.mikulskibartosz.name/using-hyperband-for-tensorflow-hyperparameter-tuning-with-keras-tuner/
# 3) https://www.mikulskibartosz.name/how-to-automaticallyselect-the-hyperparameters-of-a-resnet-neural-network/

# a keras-tuner-hez tensorflow2.0 szükséges, 
# így először telepítsük ezt,
# majd runtime újraindít

!pip install --upgrade tensorflow
!pip install --upgrade tensorflow-gpu

In [0]:
# ellenőrizzük, hogy tényleg tensorflow 2.0 van-e

import tensorflow as tf
tf.__version__

'2.0.0'

In [0]:
# és aztán telepítsük magát a keras-tuner-t is

!pip install keras-tuner

In [0]:
# tensorflow 2.0 esetén kicsit megváltozik a keras betöltése
# (mert a tensorflow részévé vált)
# de aztán a többi rész hasonló, mint eddig

import tensorflow as tf
from tensorflow import keras
import numpy as np

fashion_mnist = keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

train_images = train_images / 255.0
test_images = test_images / 255.0

train_images = train_images.reshape(len(train_images), 28, 28, 1)
test_images = test_images.reshape(len(test_images), 28, 28, 1)


In [0]:
# először készítünk egy modell-generáló függvényt,
# ami a hp. hiperparaméterekből hálót állít elő
# a hyperas-hoz hasonlóan a keras-tuner-ben is különböző képpen lehet megadni a 
# hiperparaméter tartományokat:
# - hp.Int -> egész számok egy adott tartományban szétosztva, pl. konvolúciós filterek
# - hp.Choice -> választási lehetőség egy listából, pl. optimizáló
# - hp.Float -> az Int-hez hasonlóan kell min-max tartomány, pl. dropout-hoz jó
# - hp.Boolean -> bináris döntés, pl. egy adott háló-ág szerepeljen-e
# - hp.Fixed -> ha egy paramétert nem szeretnénk változtatni. erről majd később.
# részletek itt: https://github.com/keras-team/keras-tuner/blob/master/kerastuner/engine/hyperparameters.py


# most egy konvolúciós hálót rakunk össze a fashionmnist osztályozásra

def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=64, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  

  # when using the 'sparse_categorical_crossentropy' loss, your targets should be integer targets.
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model



In [0]:
# a legegyszerűbb hiperparaméter kereső algoritmus a randomsearch.
# a nevének megfelelően véletlenül választ a paraméterek közül
# most nem foglalkozunk vele, mert van ennél érdekesebb is

from kerastuner.tuners import RandomSearch

tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    directory='output',
    project_name='FashionMNIST_random')


In [0]:
# a keras-tuner-ben egyelőre nincs implementálva a TPE

# helyette van Hyperband, ami először 'belenéz' többféle hálóba,
# és a rosszul teljesítő hálókat eldobja
# 
# azaz futtat pl. 2-2 epoch-ot különböző hiperparaméter kombinációk közül,
# (a lenti log-ban: initial_epoch: 0)
# aztán a jól teljesítők közül megint 2-2 epoch,
# (a lenti log-ban: initial_epoch: 2)
# és így tovább, iteratívan szűkíti a keresési teret
# (a lenti log-ban: initial_epoch: 4, ...)
# a vége felé pedig már végigmegy az összes epoch-on
#
# a 'factor' paraméterrel lehet szabályozni, hogy mennyire gyorsan szűkítsen,
# a 'max_epochs' pedig a nevének megfelelően max ennyi epoch-ot enged
#

from kerastuner.tuners import Hyperband

tuner = Hyperband(
    build_model,
    objective='val_accuracy',
    factor=3,
    max_epochs=10,
    directory='output',
    project_name='FashionMNIST_hyperband')

In [0]:
# először nézzük meg, hogy mi lesz a keresési terünk

tuner.search_space_summary()

In [0]:
# a tuner.search paraméterei a keras model.fit-hez hasonlóak
# ez végzi magát a hiperparaméter optimalizálást
# ha elindítjuk, kb. 15 percig fut

tuner.search(train_images, train_labels, epochs=10, validation_split=0.1)



Train on 54000 samples, validate on 6000 samples
Epoch 1/2
54000/54000 [==============================] - 11s 213us/sample - loss: 0.4147 - accuracy: 0.8478 - val_loss: 0.3175 - val_accuracy: 0.8818
Epoch 2/2
54000/54000 [==============================] - 8s 145us/sample - loss: 0.2741 - accuracy: 0.8987 - val_loss: 0.2784 - val_accuracy: 0.8987


Train on 54000 samples, validate on 6000 samples
Epoch 1/2
54000/54000 [==============================] - 9s 159us/sample - loss: 0.4275 - accuracy: 0.8452 - val_loss: 0.3296 - val_accuracy: 0.8803
Epoch 2/2
54000/54000 [==============================] - 8s 154us/sample - loss: 0.2817 - accuracy: 0.8973 - val_loss: 0.2834 - val_accuracy: 0.8947


Train on 54000 samples, validate on 6000 samples
Epoch 1/2
54000/54000 [==============================] - 8s 156us/sample - loss: 0.5337 - accuracy: 0.8104 - val_loss: 0.4557 - val_accuracy: 0.8452
Epoch 2/2
54000/54000 [==============================] - 8s 147us/sample - loss: 0.4212 - accuracy: 0.8438 - val_loss: 0.4567 - val_accuracy: 0.8382


Train on 54000 samples, validate on 6000 samples
Epoch 1/2
54000/54000 [==============================] - 8s 156us/sample - loss: 0.5444 - accuracy: 0.8147 - val_loss: 0.4000 - val_accuracy: 0.8488
Epoch 2/2
54000/54000 [==============================] - 8s 148us/sample - loss: 0.3938 - accuracy: 0.8551 - val_loss: 0.3970 - val_accuracy: 0.8467


Train on 54000 samples, validate on 6000 samples
Epoch 1/2
54000/54000 [==============================] - 8s 156us/sample - loss: 0.3972 - accuracy: 0.8571 - val_loss: 0.3030 - val_accuracy: 0.8855
Epoch 2/2
54000/54000 [==============================] - 8s 149us/sample - loss: 0.2551 - accuracy: 0.9058 - val_loss: 0.2800 - val_accuracy: 0.8948


...

Train on 54000 samples, validate on 6000 samples
Epoch 1/10
54000/54000 [==============================] - 8s 143us/sample - loss: 0.4308 - accuracy: 0.8451 - val_loss: 0.3342 - val_accuracy: 0.8742
Epoch 2/10
54000/54000 [==============================] - 7s 134us/sample - loss: 0.2937 - accuracy: 0.8924 - val_loss: 0.2819 - val_accuracy: 0.8947
Epoch 3/10
54000/54000 [==============================] - 7s 134us/sample - loss: 0.2454 - accuracy: 0.9091 - val_loss: 0.2567 - val_accuracy: 0.9063
Epoch 4/10
54000/54000 [==============================] - 7s 132us/sample - loss: 0.2089 - accuracy: 0.9215 - val_loss: 0.2685 - val_accuracy: 0.9030
Epoch 5/10
54000/54000 [==============================] - 7s 130us/sample - loss: 0.1769 - accuracy: 0.9343 - val_loss: 0.2658 - val_accuracy: 0.9030
Epoch 6/10
54000/54000 [==============================] - 7s 132us/sample - loss: 0.1508 - accuracy: 0.9432 - val_loss: 0.2711 - val_accuracy: 0.9088
Epoch 7/10
54000/54000 [===========================

Train on 54000 samples, validate on 6000 samples
Epoch 1/10
54000/54000 [==============================] - 8s 153us/sample - loss: 0.4092 - accuracy: 0.8523 - val_loss: 0.3031 - val_accuracy: 0.8885
Epoch 2/10
54000/54000 [==============================] - 8s 146us/sample - loss: 0.2636 - accuracy: 0.9043 - val_loss: 0.2656 - val_accuracy: 0.9030
Epoch 3/10
54000/54000 [==============================] - 8s 147us/sample - loss: 0.2046 - accuracy: 0.9251 - val_loss: 0.2602 - val_accuracy: 0.9080
Epoch 4/10
54000/54000 [==============================] - 8s 148us/sample - loss: 0.1606 - accuracy: 0.9406 - val_loss: 0.2569 - val_accuracy: 0.9095
Epoch 5/10
54000/54000 [==============================] - 8s 146us/sample - loss: 0.1259 - accuracy: 0.9532 - val_loss: 0.2747 - val_accuracy: 0.9098
Epoch 6/10
54000/54000 [==============================] - 8s 147us/sample - loss: 0.0948 - accuracy: 0.9649 - val_loss: 0.3159 - val_accuracy: 0.9018
Epoch 7/10
54000/54000 [===========================

Train on 54000 samples, validate on 6000 samples
Epoch 1/10
54000/54000 [==============================] - 8s 142us/sample - loss: 0.5680 - accuracy: 0.8060 - val_loss: 0.4396 - val_accuracy: 0.8405
Epoch 2/10
54000/54000 [==============================] - 7s 134us/sample - loss: 0.4075 - accuracy: 0.8518 - val_loss: 0.4260 - val_accuracy: 0.8428
Epoch 3/10
54000/54000 [==============================] - 7s 134us/sample - loss: 0.3863 - accuracy: 0.8580 - val_loss: 0.4037 - val_accuracy: 0.8510
Epoch 4/10
54000/54000 [==============================] - 7s 135us/sample - loss: 0.3707 - accuracy: 0.8641 - val_loss: 0.4184 - val_accuracy: 0.8503
Epoch 5/10
54000/54000 [==============================] - 7s 133us/sample - loss: 0.3602 - accuracy: 0.8661 - val_loss: 0.4178 - val_accuracy: 0.8543
Epoch 6/10
54000/54000 [==============================] - 7s 132us/sample - loss: 0.3470 - accuracy: 0.8706 - val_loss: 0.3971 - val_accuracy: 0.8587
Epoch 7/10
54000/54000 [===========================

Train on 54000 samples, validate on 6000 samples
Epoch 1/10
54000/54000 [==============================] - 8s 155us/sample - loss: 0.4010 - accuracy: 0.8567 - val_loss: 0.2863 - val_accuracy: 0.8993
Epoch 2/10
54000/54000 [==============================] - 8s 148us/sample - loss: 0.2519 - accuracy: 0.9065 - val_loss: 0.2929 - val_accuracy: 0.8882
Epoch 3/10
54000/54000 [==============================] - 8s 147us/sample - loss: 0.1944 - accuracy: 0.9283 - val_loss: 0.2549 - val_accuracy: 0.9085
Epoch 4/10
54000/54000 [==============================] - 8s 149us/sample - loss: 0.1453 - accuracy: 0.9464 - val_loss: 0.2721 - val_accuracy: 0.9107
Epoch 5/10
54000/54000 [==============================] - 8s 148us/sample - loss: 0.1096 - accuracy: 0.9599 - val_loss: 0.3000 - val_accuracy: 0.9083
Epoch 6/10
54000/54000 [==============================] - 8s 148us/sample - loss: 0.0812 - accuracy: 0.9696 - val_loss: 0.2827 - val_accuracy: 0.9182
Epoch 7/10
54000/54000 [===========================

INFO:tensorflow:Oracle triggered exit


In [0]:
# utána a tuner-ből kinyerhetjük a legjobb modellt, és azt használhatjuk tovább,
# pl újrataníthatunk vele

best_model = tuner.get_best_models(num_models=1)[0]
best_model.summary()

# best_model.fit(train_images, train_labels, epochs=20, validation_split=0.1, initial_epoch=4)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 48)        27696     
_________________________________________________________________
flatten (Flatten)            (None, 27648)             0         
_________________________________________________________________
dense (Dense)                (None, 112)               3096688   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1130      
Total params: 3,126,154
Trainable params: 3,126,154
Non-trainable params: 0
_________________________________________________________________


In [0]:
# a tuner-ből kinyerhető, hogy mik voltak a legjobb hiperparaméterek

params_best = tuner.get_best_hyperparameters(num_trials=1)[0]
params_best.get_config()['values']

{'conv_1_filter': 64,
 'conv_1_kernel': 3,
 'conv_2_filter': 48,
 'conv_2_kernel': 3,
 'dense_1_units': 112,
 'learning_rate': 0.001,
 'tuner/bracket': 2,
 'tuner/epochs': 10,
 'tuner/initial_epoch': 4,
 'tuner/round': 2,
 'tuner/trial_id': 'f3d57c2f47ce641d7bfb50fae5051b05'}

In [0]:
# a tuner-ből kinyerhető, hogy mik lettek a legjobb eredmények
tuner.results_summary()

In [0]:
# konklúzió: konvolúciós háló + keras-tuner + hyperband: megint sikerült picit jobb eredményt elérni

In [0]:
# a keras-tuner-ben megtalálható néhány nevezetes hálónak a hiperoptolható változata,
# konkrétan eddig Xception és ResNet van

# itt most nem előretanított hálót töltünk be, mint a transfer learning esetén,
# hanem a háló szerkezetét, amiben néhány paramétert optimalizálhatunk
# a saját adataink függvényében

# ahhoz, hogy megnézzük, mik a változtatható paraméterek, célszerű megnézni a hálók forrását,
# https://github.com/keras-team/keras-tuner/blob/master/kerastuner/applications/


In [0]:
from kerastuner.tuners import Hyperband
from kerastuner.applications import HyperResNet
from kerastuner import HyperParameters

hypermodel = HyperResNet(input_shape=(28, 28, 1), classes=10)

hp = HyperParameters()
hp.Choice('learning_rate', values=[1e-3, 1e-4])
hp.Fixed('optimizer', value='adam')

'adam'

In [0]:
tuner = Hyperband(
    hypermodel,
    objective='val_accuracy',
    hyperparameters=hp,
    tune_new_entries=False,
    max_epochs=5,
    directory='output',
    project_name='FashionMNIST_resnet')

In [0]:
# nézzük meg, hogy mi lesz a keresési terünk

# mivel az optimizer-t fix-re állítottuk, azt nem fogja változtatni,
# és a 'tune_new_entries=False' miatt a többi hiperparamétert sem piszkálja
# csak a két learning_rate értéket fogja megnézni

tuner.search_space_summary()

In [0]:
# ha beállítjuk, hogy 'tune_new_entries=False', akkor
# viszont a háló többi paraméterét is végignézné

tuner_large = Hyperband(
    hypermodel,
    objective='val_accuracy',
    hyperparameters=hp,
    # ez most True
    tune_new_entries=True,
    max_epochs=5,
    directory='output',
    project_name='FashionMNIST_resnet')

tuner_large.search_space_summary()

In [0]:
# a ResNet-hez a címkéket onehot-enkódolni kell

from keras.utils import to_categorical
train_labels_binary = to_categorical(train_labels)

Using TensorFlow backend.


In [0]:
# itt most csak 2 tanítás megy végig a 2-féle learning rate-tel
# de mivel a ResNet hálózat nagy, ezért sokáig tart

tuner.search(train_images, train_labels_binary, validation_split=0.1)

Train on 54000 samples, validate on 6000 samples
Epoch 1/2
54000/54000 [==============================] - 83s 2ms/sample - loss: 0.5668 - accuracy: 0.8071 - val_loss: 0.4473 - val_accuracy: 0.8313
Epoch 2/2
54000/54000 [==============================] - 70s 1ms/sample - loss: 0.4696 - accuracy: 0.8421 - val_loss: 0.3488 - val_accuracy: 0.8702


Train on 54000 samples, validate on 6000 samples
Epoch 1/2
54000/54000 [==============================] - 79s 1ms/sample - loss: 0.6895 - accuracy: 0.7522 - val_loss: 0.4961 - val_accuracy: 0.8193
Epoch 2/2
54000/54000 [==============================] - 71s 1ms/sample - loss: 0.4285 - accuracy: 0.8436 - val_loss: 0.4022 - val_accuracy: 0.8550


INFO:tensorflow:Oracle triggered exit


In [0]:
tuner.results_summary()

In [0]:
params_best = tuner.get_best_hyperparameters(num_trials=1)[0]
params_best.get_config()['values']

{'conv3_depth': 4,
 'conv4_depth': 6,
 'learning_rate': 0.001,
 'optimizer': 'adam',
 'pooling': 'avg',
 'tuner/bracket': 1,
 'tuner/epochs': 2,
 'tuner/initial_epoch': 0,
 'tuner/round': 0,
 'version': 'v2'}

In [0]:
# a legjobb modellt visszaállíthatjuk a hypermodel és a params_best kombinációjából
model_best = tuner.hypermodel.build(params_best)

In [0]:
print(model_best.summary())

Model: "ResNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 34, 34, 1)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 14, 14, 64)   3200        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 16, 16, 64)   0           conv1_conv[0][0]                 
_____________________________________________________________________________________________

In [0]:
# itt a gyakorlat vége
# konklúzió a keras-tuner-ről: most is jól működik, 
# és a következő hónapokban sok fejlesztés várható 